In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
! pip install navec
! pip install slovnet
! pip install pandas numpy scikit-learn torch tqdm transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
import random
import string
import nltk
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
from navec import Navec
from slovnet.model.emb import NavecEmbedding

from tqdm.notebook import tqdm
from transformers import MarianMTModel, MarianTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer
tqdm.pandas()

# Подгружаем данные

In [ ]:
file_path = 'LK_modified.xlsx'
all_sheets = pd.read_excel(file_path, sheet_name=None)

dfs = {sheet_name: pd.DataFrame(sheet_data) for sheet_name, sheet_data in all_sheets.items()}


qa_df = dfs[list(dfs.keys())[0]]
sintetic = dfs[list(dfs.keys())[1]]
popular_phrases = dfs[list(dfs.keys())[2]]
glossary = dfs[list(dfs.keys())[3]]

In [ ]:
qa_df

,id,question,content,category,source
0,0,"Я сменил автомобить, на учет еще не поставил, ...",Для внесения данных по личному автомобилю обра...,автомобиль,оригинал
1,1,Не отображается автомобиль в личном кабинете.,Для внесения данных по личному автомобилю обра...,автомобиль,оригинал
2,2,добавить автомобиль,Для внесения данных по личному автомобилю обра...,автомобиль,оригинал
3,3,хочу внести данные об автомобиле,Для внесения данных по личному автомобилю обра...,автомобиль,оригинал
4,4,Как внести данные об автомобиле?,Для внесения данных по личному автомобилю обра...,автомобиль,оригинал
...,...,...,...,...,...
5784,5784,как сделать продление срочного трудового договора,Для продления срочного трудового договора созд...,график работы,синтетика
5785,5785,как продлить договор,Для продления срочного трудового договора созд...,график работы,синтетика
5786,5786,как перевести сотрудника на бессрочный ТД,Для продления срочного трудового договора созд...,график работы,синтетика
5787,5787,как продлить трудовой договор?,Для продления срочного трудового договора созд...,график работы,синтетика


In [ ]:
glossary

,Сокращение,Расшифровка
0,лк,личный кабинет
1,БиР,Беременность и роды
2,зп,заработная плата
3,НДФЛ,Налог на доходы физических лиц
4,СТД,срочный трудовой договор
5,ТК,трудовой договор
6,АО,авансовый отчет
7,SLA,сроки
8,ЭЦП,электронная цифровая подпись
9,КР,кадровый резерв


# Смотрим на данные

In [ ]:
proportions = qa_df['category'].value_counts(dropna=False).reset_index()
print(proportions.head(5), '\n')
print(proportions.tail(10))

           category  count
0            отпуск    967
1                ЛК    800
2               БиР    719
3  удаленная работа    386
4   прием на работу    317 

           category  count
22  документооборот     22
23         оператор     21
24              МЧД     16
25    выручай-карта     15
26               СБ     14
27              SED     10
28  уход за больным      9
29       автомобиль      8
30         обучение      7
31           Отпуск      1


# Добавляем объяснение сокращения по глоссарию

In [ ]:
qa_df['question'] = qa_df['question'].str.lower()
qa_df['category'] = qa_df['category'].str.lower()

replacement_dict = pd.Series(glossary['Расшифровка'].values, index=glossary['Сокращение'].str.lower()).to_dict()

def replace_abbreviations(text, replacements):
    for abbrev, full in replacements.items():
        text = text.replace(abbrev.lower(), full)
    return text

qa_df['question'] = qa_df['question'].apply(lambda x: replace_abbreviations(x, replacement_dict))

In [ ]:
qa_df[qa_df['id']==137] #ЗП на заработную плату

,id,question,content,category,source
137,137,в каких числа приходит заработная плата?,Заработная плата в нашей компании выплачиваетс...,зарплата,оригинал


# Конкатим категорию к началу каждого вопроса

In [ ]:
### тут перенесла на после аугментации

# Кодируемся

In [ ]:
def encode_column(df, column_name):
  id_map = {}
  for x in df[column_name]:
    id_map[x] = id_map.get(x, len(id_map))
  df[f'{column_name}_id'] = df[column_name].map(id_map)
  return id_map

In [ ]:
id_map_content = encode_column(qa_df, 'content')
id_map_categories = encode_column(qa_df, 'category')

In [ ]:
qa_df['content_count'] = qa_df['content_id'].map(qa_df.groupby('content_id').agg('size'))

# Функции для аугменации

In [ ]:
def add_or_remove_punctuation(text):
    """Добавление или удаление знаков препинания."""
    # Возможные варианты добавления знаков препинания
    punctuations = [',', '.', '!', '?']
    words = text.split()

    # Добавляем или удаляем знаки препинания
    if random.random() < 0.5:
        # Добавить знак препинания
        position = random.randint(0, len(words) - 1)
        punct = random.choice(punctuations)
        words[position] = words[position] + punct
    else:
        # Удалить знак препинания, если он есть
        text = text.translate(str.maketrans('', '', string.punctuation))
        words = text.split()

    return ' '.join(words)

In [ ]:
def introduce_typo(text):
    """Создание опечаток в тексте."""
    if not text:
        return text

    words = text.split()
    index = random.randint(0, len(words) - 1)
    word = words[index]

    # Опечатки: замена, пропуск или дублирование символов
    typo_type = random.choice(['swap', 'remove', 'duplicate'])

    if typo_type == 'swap' and len(word) > 1:
        # Меняем местами соседние буквы
        pos = random.randint(0, len(word) - 2)
        word = list(word)
        word[pos], word[pos + 1] = word[pos + 1], word[pos]
        words[index] = ''.join(word)

    elif typo_type == 'remove' and len(word) > 1:
        # Удаляем случайную букву
        pos = random.randint(0, len(word) - 1)
        words[index] = word[:pos] + word[pos + 1:]

    elif typo_type == 'duplicate':
        # Дублируем случайную букву
        pos = random.randint(0, len(word) - 1)
        words[index] = word[:pos] + word[pos] + word[pos:]

    return ' '.join(words)

In [ ]:
def shuffle_words(text):
    """Перестановка порядка слов."""
    words = text.split()
    if len(words) > 1:
        random.shuffle(words)
    return ' '.join(words)

In [ ]:
model_name = "cointegrated/rut5-base-paraphraser"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def generate_paraphrases(text, num_return_sequences=3, num_beams=5):
    input_text = "" + text + " </s>"
    encoding = tokenizer.encode_plus(input_text, padding=True, return_tensors="pt", max_length=128, truncation=True)
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=128,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

    return [tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in outputs]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

In [ ]:
src_lang = "Helsinki-NLP/opus-mt-ru-en"
tgt_lang = "Helsinki-NLP/opus-mt-en-ru"

tokenizer_src = MarianTokenizer.from_pretrained(src_lang)
model_src = MarianMTModel.from_pretrained(src_lang)

tokenizer_tgt = MarianTokenizer.from_pretrained(tgt_lang)
model_tgt = MarianMTModel.from_pretrained(tgt_lang)

def backtranslate_with_temperature(sentence, num_variations=5, temperature=1):
    inputs = tokenizer_src(sentence, return_tensors="pt")
    translated = model_src.generate(**inputs, temperature=temperature, num_return_sequences=num_variations)

    generated_sentences = []
    for translation in translated:
        translated_sentence = tokenizer_src.decode(translation, skip_special_tokens=True)
        inputs_back = tokenizer_tgt(translated_sentence, return_tensors="pt")
        back_translated = model_tgt.generate(**inputs_back, temperature=temperature)
        final_sentence = tokenizer_tgt.decode(back_translated[0], skip_special_tokens=True)
        generated_sentences.append(final_sentence)

    return generated_sentences



tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

...

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

# Пример аугментации

In [ ]:
# Пример использования
text = "всем привет, динозавры"
augmented_text = add_or_remove_punctuation(text)
augmented_text = introduce_typo(augmented_text)
augmented_text = shuffle_words(augmented_text)

print("Оригинальное предложение:", text)
print("Аугментированное предложение:", augmented_text)

Оригинальное предложение: всем привет, динозавры
Аугментированное предложение: дниозавры привет всем


In [ ]:
augmented_text = generate_paraphrases(text)
print("Оригинальное предложение:", text)
print("Аугментированное предложение:", augmented_text)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Оригинальное предложение: всем привет, динозавры
Аугментированное предложение: ['Привет всем, динозавры.', 'Всем привет, динозавры.', 'Привет всем, динозавры!']


In [ ]:
augmented_text = backtranslate_with_temperature(text, num_variations=5, temperature=1.3)
print("Оригинальное предложение:", text)
print("Аугментированное предложение:", augmented_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Оригинальное предложение: всем привет, динозавры
Аугментированное предложение: ['Привет, динозавры.', 'Всем привет, динозавры.', 'Всем привет, динозавры.', 'Привет, динозавры.', 'Привет, динозавры.']


# Расширение датасета

In [ ]:
import random

AUG_NUM = 30

def balance_dataset(qa_df, augmentation_method='paraphrase'):
    # Шаг 1: Найти самый частовстречаемый ответ
    max_count = qa_df['content_id'].value_counts().max()

    # Шаг 2: Сбалансировать выборку ответов
    augmented_data = []
    for content_id, group in qa_df.groupby('content_id'):
        count = len(group)
        augmented_data.extend(group.to_dict('records'))  # Добавляем все исходные строки

        # Если ответ встречается реже, чем самый частовстречаемый, создаем аугментированные копии вопросов
        aug_count = 0  # Счетчик для текущего числа аугментаций
        while aug_count < min(AUG_NUM, max_count - count):
            row = group.sample(1).iloc[0].to_dict()  # Случайный вопрос из группы
            question = row['question']

            # Выбираем метод аугментации на основе параметра
            if augmentation_method == 'paraphrase':
                augmented_questions = generate_paraphrases(question, num_return_sequences=5)
            elif augmentation_method == 'backtranslation':
                augmented_questions = backtranslate_with_temperature(question, num_variations=5, temperature=1.3)
            elif augmentation_method == 'punctuation_typo_shuffle':
                augmented_question = add_or_remove_punctuation(question)
                augmented_question = introduce_typo(augmented_question)
                augmented_questions = [shuffle_words(augmented_question)]  # создаем список с одним элементом
            else:
                raise ValueError("Unknown augmentation method selected.")

            # Если метод возвращает список, добавляем все элементы
            for augmented_question in augmented_questions:
                if aug_count >= min(AUG_NUM, max_count - count):
                    break
                new_row = row.copy()
                new_row['question'] = augmented_question
                augmented_data.append(new_row)
                aug_count += 1

    # Шаг 3: Создать новый сбалансированный датафрейм
    balanced_qa_df = pd.DataFrame(augmented_data)
    return balanced_qa_df

In [ ]:
balance_dataset(qa_df[:10], augmentation_method='punctuation_typo_shuffle')

,id,question,content,category,source,content_id,category_id,content_count
0,0,"я сменил автомобить, на учет еще не поставил, ...",Для внесения данных по личному автомобилю обра...,автомобиль,оригинал,0,0,8
1,1,не отображается автомобиль в личном кабинете.,Для внесения данных по личному автомобилю обра...,автомобиль,оригинал,0,0,8
2,2,добавить автомобиль,Для внесения данных по личному автомобилю обра...,автомобиль,оригинал,0,0,8
3,3,хочу внести данные об автомобиле,Для внесения данных по личному автомобилю обра...,автомобиль,оригинал,0,0,8
4,4,как внести данные об автомобиле?,Для внесения данных по личному автомобилю обра...,автомобиль,оригинал,0,0,8
5,5,мне нужно внести данные об автомобиле,Для внесения данных по личному автомобилю обра...,автомобиль,оригинал,0,0,8
6,6,внести машину,Для внесения данных по личному автомобилю обра...,автомобиль,оригинал,0,0,8
7,7,"при смене автомобиля, который используется в с...",Для внесения данных по личному автомобилю обра...,автомобиль,оригинал,0,0,8
8,8,"будет ли оплачен больничный, если он отрудовой...",Сотруднику следует обратиться в мед. организац...,бир,оригинал,1,1,15
9,8,"отрудовой? если он в будет больничный, ли дого...",Сотруднику следует обратиться в мед. организац...,бир,оригинал,1,1,15


# Модели эмбеддингов

In [ ]:
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'  # 51MB
navec = Navec.load(path)  # ~1 sec, ~100MB RAM

emb = NavecEmbedding(navec)

# Функция для преобразования текста в индексы
def text_to_indices(text, navec):
    tokens = text.split()  # Используем пробелы для токенизации
    indices = [navec.vocab.get(token, navec.vocab['<unk>']) for token in tokens]
    return indices

# Функция для получения эмбеддингов
def get_navec_embeddings(text, emb, navec):
    indices = text_to_indices(text, navec)
    if not indices:
        return torch.zeros(1, emb.embedding_dim)  # Возвращаем нулевой тензор для пустых запросов
    input_tensor = torch.tensor(indices)
    embeddings = emb(input_tensor).mean(dim=0)  # Среднее значение по всем токенам
    return embeddings

/usr/local/lib/python3.10/dist-packages/slovnet/model/emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  torch.from_numpy(navec.pq.indexes),


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd

# Инициализация токенизатора и модели RuBERT
tokenizer_Bert = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model_Bert = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')

# Функция для получения эмбеддингов текста
def get_embeddings_bert(texts):
    inputs = tokenizer_Bert(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model_Bert(**inputs)
    # Используем последние скрытые состояния в качестве эмбеддингов
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Усреднение по всем токенам
    return embeddings

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import GPT2Tokenizer, GPT2Model

# Инициализация токенизатора и модели RuGPT-3
tokenizer_gpt = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
model_gpt = GPT2Model.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

# Функция для получения эмбеддингов текста с помощью RuGPT-3
def get_embeddings_gpt3(texts):
    inputs = tokenizer_gpt(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model_gpt(**inputs)
    # Используем последние скрытые состояния в качестве эмбеддингов
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Усреднение по всем токенам
    return embeddings

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

# Пайплайн предобработки

In [ ]:
def process_splitted_data(df, do_aug=True, augmentation_method='paraphrase', encoding_method='navec'):
    augmented_df = df.copy()
    if do_aug:
        augmented_df = balance_dataset(df, augmentation_method=augmentation_method)

    augmented_df['question'] = augmented_df['category'] + " " + augmented_df['question']
    augmented_df['question'] = augmented_df['question'].str.lower()

    if encoding_method == 'navec':
        augmented_df['embeddings'] = augmented_df['question'].apply(lambda x: get_navec_embeddings(x, emb, navec).tolist())
        augmented_df['embeddings'] = augmented_df['embeddings'].apply(np.array)

    elif encoding_method == 'sber-gpt':
        augmented_df['embeddings'] = augmented_df['question'].progress_apply(lambda x: get_embeddings_gpt3([x])[0].numpy())

    elif encoding_method == 'rubert':
        augmented_df['embeddings'] = augmented_df['question'].progress_apply(lambda x: get_embeddings_bert([x])[0].numpy())

    else:
        raise ValueError(f"Unsupported encoding method: {encoding_method}")

    X = np.stack(augmented_df['embeddings'].values)
    y = augmented_df['content_id']
    return X, y

In [ ]:
IS_XGBOOST = False

# на случай если буду использовать другие модельки
def format_data(data):
  if IS_XGBOOST:
    return xgb.DMatrix(data)
  else:
    return data

# Дальше - если тестить без кроссвалидации (для моделек потяжелее чем линейная регрессия)

In [ ]:
from sklearn.model_selection import train_test_split

def stratified_train_test_split(df, content_column, source_column, test_size=0.2, random_state=42, with_synthetic=True):
    """
    Разделение данных на обучающую и тестовую выборки с учетом следующих шагов:
    1) Если with_synthetic=True:
        - Оригинальные данные из source_column.
        - Наблюдения, которые встречаются 1 раз по content_column, откладываются отдельно.
        - Стратифицированное разделение на train/test.
        - Единичные наблюдения добавляются в тест.
        - Синтетические данные добавляются в train.
    2) Если with_synthetic=False:
        - Фильтрация данных от частотных наблюдений (>= 3).
        - Стратифицированное базовое разделение на train/test.
    """

    if with_synthetic:
        # Шаги для работы с синтетикой
        original_data = df[df[source_column] == 'оригинал']
        content_counts = original_data[content_column].value_counts()
        single_content = content_counts[content_counts == 1].index
        single_data = original_data[original_data[content_column].isin(single_content)]

        remaining_data = original_data[~original_data[content_column].isin(single_content)]

        train_data, test_data = train_test_split(remaining_data, test_size=test_size,
                                                 stratify=remaining_data[content_column],
                                                 random_state=random_state)

        X_test = pd.concat([test_data, single_data])

        synthetic_data = df[df[source_column] == 'синтетика']
        X_train = pd.concat([train_data, synthetic_data])

    else:
        # Шаги для работы без синтетики
        original_data = df[df[source_column] == 'оригинал'].copy()
        original_data['content_count'] = original_data['content_id'].map(original_data['content_id'].value_counts())
        filtered_df = original_data[original_data['content_count'] >= 3]

        X_train, X_test = train_test_split(filtered_df, test_size=test_size,
                                           stratify=filtered_df[content_column],
                                           random_state=random_state)

    y_train = X_train['content_id']
    y_test = X_test['content_id']

    return X_train, X_test, y_train, y_test


In [ ]:
def get_metrics(y_true, y_pred):
  # Подсчет метрик
    accuracy = accuracy_score(y_true, y_pred)
    precision_macro = precision_score(y_true, y_pred, average='macro')
    precision_micro = precision_score(y_true, y_pred, average='micro')
    recall_macro = recall_score(y_true, y_pred, average='macro')
    recall_micro = recall_score(y_true, y_pred, average='micro')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_micro = f1_score(y_true, y_pred, average='micro')

    # Вывод метрик
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: (macro:{precision_macro:.4f}, micro:{precision_micro:.4f})')
    print(f'Recall: (macro:{recall_macro:.4f}, micro:{recall_micro:.4f})')
    print(f'F1 Score: (macro:{f1_macro:.4f}, micro:{f1_micro:.4f}) \n')

In [ ]:
# X_train, X_test, y_train, y_test = stratified_train_test_split(qa_df, content_column='content', source_column='source', test_size=0.5, random_state=42, with_synthetic=True) # ставить для синтетики тест сайз больше

### Исходно без аугментаций

In [ ]:
X_train, X_test, y_train, y_test = stratified_train_test_split(qa_df, content_column='content', source_column='source', test_size=0.25, random_state=42, with_synthetic=False)

In [ ]:
X_train

,id,question,content,category,source,content_id,category_id,content_count
1446,1446,руководитель не хочет увольнять,Если решить вопрос по увольнению/переводу с не...,увольнение,оригинал,174,23,7
1526,1526,продлите заявку на ур,Данная заявка была завершена т.к истек срок на...,удаленная работа,оригинал,184,24,4
1251,1251,не могу сделать заявку на сотрудника,"Если в ""команде"" нет подчиненных сотрудников п...",табель,оригинал,157,22,149
599,599,новому сотруднику не пришел логин и пароль для...,"При проблемах со входом в личный кабинет, преж...",лк,оригинал,70,8,284
1567,1567,оплата больничного листа за уход за ребенком,Больничный с кодом 09 (уход за больным членом ...,уход за больным,оригинал,188,25,9
...,...,...,...,...,...,...,...,...
283,283,изменить номер телефона в личный кабинет сотру...,"Кнопка ""изменить номер"" телефона находится в л...",лк,оригинал,66,8,55
512,512,нет возможности зайти в личный кабинет- не вер...,"При проблемах со входом в личный кабинет, преж...",лк,оригинал,70,8,284
1008,1008,не могу отработать заявку,"Создайте, пожалуйста, обращение в ИТ поддержку...",поддержка,оригинал,79,16,275
565,565,востановить личный кабинет,"При проблемах со входом в личный кабинет, преж...",лк,оригинал,70,8,284


In [ ]:

# backtranslation
# encoding_method = 'navec', 'sber-gpt', 'rubert'
# augmentation_method = 'paraphrase', 'punctuation_typo_shuffle', 'backtranslation'
X_train_processed, y_train_processed = process_splitted_data(X_train, encoding_method='rubert',
                                                             augmentation_method='backtranslation')
X_test_no_aug, y_test_no_aug = process_splitted_data(X_test, do_aug=False, encoding_method='rubert')
print(X_train_processed.shape)
print(X_test_no_aug.shape)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

  0%|          | 0/4072 [00:00<?, ?it/s]

  0%|          | 0/385 [00:00<?, ?it/s]

(4072, 768)
(385, 768)


In [ ]:
model_lr = LogisticRegression(max_iter=1000, random_state=42)
model_lr.fit(X_train_processed, y_train_processed)

IS_XGBOOST = False


def format_data(data):
    if IS_XGBOOST:
        return xgb.DMatrix(data)
    else:
        return data


y_pred_train = model_lr.predict(format_data(X_train_processed))
y_pred_test_no_aug = model_lr.predict(format_data(X_test_no_aug))
get_metrics(y_train_processed, y_pred_train)

Accuracy: 0.9980
Precision: (macro:0.9979, micro:0.9980)
Recall: (macro:0.9979, micro:0.9980)
F1 Score: (macro:0.9978, micro:0.9980) 


In [ ]:
get_metrics(y_test_no_aug, y_pred_test_no_aug)

Accuracy: 0.7844
Precision: (macro:0.5949, micro:0.7844)
Recall: (macro:0.6374, micro:0.7844)
F1 Score: (macro:0.6001, micro:0.7844) 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# encoding_method = 'navec', 'sber-gpt', 'rubert'
# augmentation_method = 'paraphrase', 'punctuation_typo_shuffle', 'backtranslation'
print(X_train.shape)
print(X_test.shape)
X_train_processed, y_train_processed = process_splitted_data(X_train, do_aug=False, encoding_method='rubert')
X_test_no_aug, y_test_no_aug = process_splitted_data(X_test, do_aug=False, encoding_method='rubert')
print(X_train_processed.shape)
print(X_test_no_aug.shape)

(1155, 8)
(385, 8)


  0%|          | 0/1155 [00:00<?, ?it/s]

  0%|          | 0/385 [00:00<?, ?it/s]

(1155, 768)
(385, 768)


In [ ]:
model_lr = LogisticRegression(max_iter=1000, random_state=42)
model_lr.fit(X_train_processed, y_train_processed)

IS_XGBOOST = False


def format_data(data):
    if IS_XGBOOST:
        return xgb.DMatrix(data)
    else:
        return data


y_pred_train = model_lr.predict(format_data(X_train_processed))
y_pred_test_no_aug = model_lr.predict(format_data(X_test_no_aug))
get_metrics(y_train_processed, y_pred_train)

Accuracy: 1.0000
Precision: (macro:1.0000, micro:1.0000)
Recall: (macro:1.0000, micro:1.0000)
F1 Score: (macro:1.0000, micro:1.0000) 


In [ ]:
get_metrics(y_test_no_aug, y_pred_test_no_aug)

Accuracy: 0.7766
Precision: (macro:0.5897, micro:0.7766)
Recall: (macro:0.5965, micro:0.7766)
F1 Score: (macro:0.5759, micro:0.7766) 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
